In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from random import randint
import matplotlib.pyplot as plt

In [2]:
class NetGirl(nn.Module):
    def __init__(self, input_dim, num_hidden, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, num_hidden)
        self.layer2 = nn.Linear(num_hidden, output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = F.tanh(x)
        x = self.layer2(x)
        x = F.tanh(x)
        return x

In [4]:
model = NetGirl(3, 2, 1)
print(model)
print(list(model.parameters()))

# обучающая выборка (она же полная выборка)
x_train = torch.FloatTensor([(-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
                            (1, -1, -1), (1, -1, 1), (1, 1, -1), (1, 1, 1)])
y_train = torch.FloatTensor([-1, 1, -1, 1, -1, 1, -1, -1]).unsqueeze(1)  # Reshape to [8, 1]
total = len(y_train)


print(x_train.shape)
print(y_train.shape)

NetGirl(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)
[Parameter containing:
tensor([[ 0.3469, -0.3591,  0.2624],
        [-0.1840, -0.0843,  0.3818]], requires_grad=True), Parameter containing:
tensor([-0.2679,  0.3504], requires_grad=True), Parameter containing:
tensor([[0.0326, 0.1305]], requires_grad=True), Parameter containing:
tensor([-0.0520], requires_grad=True)]
torch.Size([8, 3])
torch.Size([8, 1])


In [ ]:

optimizer = optim.RMSprop(params=model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

model.train()

for _ in range(1000):
    k = randint(0, total-1)
    y = model(x_train[k])
    loss = loss_func(y, y_train[k])

    print(f"Итерация {_+1}, номер {k+1}, ошибка: {loss.data.item():.4f}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model.eval()

# тестирование обученной НС
for x, d in zip(x_train, y_train):
    with torch.no_grad():
        y = model(x)
        print(f"Выходное значение НС: {y.data.item():.4f} => {d.item():.4f}")